In [1]:
import os
import sys
import time
import logging
import json
import numpy as np
import pickle
import pdb
import pandas as pd
import requests
import re
import random
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation,SpatialDropout1D,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import BatchNormalization
from keras.regularizers import L1L2
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.models import load_model
from sklearn.model_selection import train_test_split
np.random.seed(42)


Using TensorFlow backend.


In [2]:
def read_text_file(file_name):
    data_list  = []
    with open(file_name,'r') as f:
        for line in f:
            line = line.strip()
            label = ' '.join(line[:line.find("]")].strip().split())
            text = line[line.find("]")+1:].strip()
            data_list.append([label, text])

    return data_list

In [3]:
def extract_labels(text_list):
    label_list = []
    text_list = [text_list[i][0].replace('[','') for i in range(len(text_list))]
    label_list = [list(np.fromstring(text_list[i], dtype=float, sep=' ')) for i in range(len(text_list))]
    return label_list

In [4]:
def extract_text_msgs(text_list):
    msg_list = []
    msg_list = [text_list[i][1] for i in range(len(text_list))]
    return msg_list


In [5]:
def read_glove_vector(glove_file):
    with open(glove_file,'r',encoding='UTF-8') as file:
        words = set()
        word_to_vec = {}
        for line in file:
            line = line.strip().split()
            line[0] = re.sub('[^a-zA-Z]', '', line[0])
            if len(line[0]) > 0:
                words.add(line[0])
                try:
                    word_to_vec[line[0]] = np.array(line[1:],dtype=np.float64)
                except:
                    print('Error has occured')
                    print('-'*50)
                    print(line[1:])
        i = 1
        word_to_index = {}
        index_to_word = {}
        for word in sorted(words):
            word_to_index[word] = i
            index_to_word[i] = word
            i = i+1
    #print(word_to_index,index_to_word,word_to_vec)
    return word_to_index,index_to_word,word_to_vec

In [6]:
def sentences_to_indices(text_arr,word_to_index,max_len):
    arr_len = text_arr.shape[0]
    arr_indices = np.zeros((arr_len,max_len))
    for i in range(arr_len):
        sentence = text_arr[i].lower().split()
        j = 0
        for word in sentence:
            if word in word_to_index:
                arr_indices[i,j] = word_to_index[word]
                j = j+1
    return arr_indices

In [7]:
def create_embedding_layer(word_to_index,word_to_vec):
    corpus_len = len(word_to_index) + 1
    #print( "w2v  => " , word_to_vec)
    embed_dim = word_to_vec['word'].shape[0]
    
    embed_matrix = np.zeros((corpus_len,embed_dim))

    for word, index in word_to_index.items():
        embed_matrix[index,:] = word_to_vec[word]

    embedding_layer = Embedding(corpus_len, embed_dim)
    embedding_layer.build((None,))
    embedding_layer.set_weights([embed_matrix])

    return embedding_layer


In [8]:
def create_lstm_model(input_shape,embedding_layer):
    sentence_indices = Input(shape=input_shape, dtype=np.int32)
    embedding_layer =  embedding_layer
    embeddings = embedding_layer(sentence_indices)
    reg = L1L2(0.01, 0.01)
    
    X = Bidirectional(LSTM(128, return_sequences=True,bias_regularizer=reg,kernel_initializer='he_uniform'))(embeddings)
    X = BatchNormalization()(X)
    X = Dropout(0.5)(X)
    X = LSTM(64)(X)
    X = Dropout(0.5)(X)
    X = Dense(7, activation='softmax')(X)
    X =  Activation('softmax')(X)
    model = Model(sentence_indices, X)
    return model

In [9]:
textlist = read_text_file("data.txt")

In [10]:
label_list = extract_labels(textlist)
msg_list = extract_text_msgs(textlist)

In [12]:
word_to_index,index_to_word,word_to_vec = read_glove_vector('../glove.6B/glove.6B.50d.txt')

In [14]:
x_train, x_test, y_train, y_test = train_test_split(msg_list, label_list,stratify = label_list,test_size = 0.2, random_state = 123)

In [15]:
tk = Tokenizer(lower = True, filters='')
tk.fit_on_texts(msg_list)
train_tokenized = tk.texts_to_sequences(x_train)
test_tokenized = tk.texts_to_sequences(x_test)

In [16]:
maxlen = 50

In [17]:
X_train = pad_sequences(train_tokenized, maxlen = maxlen)
X_test = pad_sequences(test_tokenized, maxlen = maxlen)

In [19]:
if os.path.exists('tokenizer.pickle'):
    os.remove('tokenizer.pickle')
    with open('tokenizer.pickle', 'wb') as tokenizer:
        pickle.dump(tk, tokenizer, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
embedding_layer = create_embedding_layer(word_to_index,word_to_vec)
model = create_lstm_model((maxlen,),embedding_layer)
print(model.summary())






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 50)            17090100  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 256)           183296    
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 256)           1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                82176     
_____

In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
model.fit(X_train, np.array(y_train), epochs = 30, batch_size = 32, shuffle=True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/30
5984/5984 [==============================] - 75s 12ms/step - loss: 6.3894 - acc: 0.1673
Epoch 2/30
5984/5984 [==============================] - 73s 12ms/step - loss: 5.1569 - acc: 0.2390
Epoch 3/30
5984/5984 [==============================] - 83s 14ms/step - loss: 4.1341 - acc: 0.3120
Epoch 4/30
5984/5984 [==============================] - 82s 14ms/step - loss: 3.3063 - acc: 0.3702
Epoch 5/30
5984/5984 [==============================] - 82s 14ms/step - loss: 2.6407 - acc: 0.4313
Epoch 6/30
5984/5984 [==============================] - 84s 14ms/step - loss: 2.1192 - acc: 0.4871
Epoch 7/30
5984/5984 [==============================] - 82s 14ms/step - loss: 1.7359 - acc: 0.5279
Epoch 8/30
5984/5984 [==============================] - 85s 14ms/step - loss: 1.6045 - acc: 0.5702
Epoch 9/30
5984/5984 [==============================] - 82s 14ms/step - loss: 1.5624 - acc: 0.6110
Epoch 10/30
598

In [23]:
model.save('emoji_model.h5')

In [25]:
loss, acc = model.evaluate(X_test, np.array(y_test))
print(loss ,acc)

1496/1496 [==============================] - 6s 4ms/step
1.680861423997318 0.48462566844919786


In [26]:
test_sent = tk.texts_to_sequences(['Feeling sad that my favourite cricketer has retired'])
test_sent = pad_sequences(test_sent, maxlen = maxlen)
pred = model.predict(test_sent)
print(np.argmax(pred))

3


In [27]:
model = load_model('emoji_model.h5' , compile = False)

In [29]:
test_sent = tk.texts_to_sequences(['fuck you'])
test_sent = pad_sequences(test_sent, maxlen = maxlen)
pred = model.predict(test_sent)
print(np.argmax(pred))

3
